* 本手册，用于对大模型的样例，学习。
* 模型调用匀使用轨迹流动模型API

# 一、思维链的示例

In [1]:
# %pip install -q requests python-dotenv
import os
import json
import requests
from typing import Iterable, List, Optional, Union
from dataclasses import dataclass
from dotenv import load_dotenv, find_dotenv
from urllib3.util.retry import Retry
from IPython.display import display, Markdown
from requests.adapters import HTTPAdapter

API_URL = "https://api.siliconflow.cn/v1/chat/completions"

def _build_messages(system: Optional[Union[str, List[str]]] = None,
                    user: Optional[Union[str, List[str]]] = None) -> List[dict]:
    msgs: List[dict] = []
    if system:
        if isinstance(system, str):
            msgs.append({"role": "system", "content": system})
        else:
            msgs.extend({"role": "system", "content": s} for s in system)
    if user:
        if isinstance(user, str):
            msgs.append({"role": "user", "content": user})
        else:
            msgs.extend({"role": "user", "content": u} for u in user)
    return msgs

@dataclass
class SiliconFlowClient:
    api_key: Optional[str] = None
    dotenv_path: Optional[str] = None
    timeout: int = 60
    max_retries: int = 3
    backoff_factor: float = 0.5

    def __post_init__(self):
        # 1) 若指定了 .env 路径则加载；否则尝试自动发现
        if self.dotenv_path:
            load_dotenv(self.dotenv_path, override=False)
        else:
            path = find_dotenv(filename=".env", usecwd=True)
            if path:
                load_dotenv(path, override=False)
        # 2) 密钥优先级：显式参数 > 环境变量
        self.api_key = self.api_key or os.getenv("SILICONFLOW_API_KEY")
        if not self.api_key:
            raise RuntimeError("未发现 API 密钥，请传入 api_key 或设置环境变量 SILICONFLOW_API_KEY 或 .env。")

        # 3) 复用会话 + 重试
        self.sess = requests.Session()
        retry = Retry(
            total=self.max_retries,
            backoff_factor=self.backoff_factor,
            status_forcelist=(429, 500, 502, 503, 504),
            allowed_methods=frozenset(["POST"]),
            raise_on_status=False,
        )
        adapter = HTTPAdapter(max_retries=retry)
        self.sess.mount("http://", adapter)
        self.sess.mount("https://", adapter)

        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }

    def complete(self,
                 model: str,
                 messages: Optional[List[dict]] = None,
                 system: Optional[Union[str, List[str]]] = None,
                 user: Optional[Union[str, List[str]]] = None,
                 temperature: float = 0.7,
                 max_tokens: Optional[int] = None) -> str:
        if messages is None:
            messages = _build_messages(system, user)
        payload = {
            "model": model,
            "messages": messages,
            "temperature": temperature,
            "stream": False,
        }
        if max_tokens is not None:
            payload["max_tokens"] = max_tokens

        resp = self.sess.post(API_URL, headers=self.headers, json=payload, timeout=self.timeout)
        resp.encoding = "utf-8"
        if not resp.ok:
            # 尽量展示服务端错误信息
            try:
                detail = resp.json()
            except Exception:
                detail = resp.text
            raise RuntimeError(f"HTTP {resp.status_code}: {detail}")

        data = resp.json()
        return (data.get("choices") or [{}])[0].get("message", {}).get("content", "")

    def stream_text(self,
                    model: str,
                    messages: Optional[List[dict]] = None,
                    system: Optional[Union[str, List[str]]] = None,
                    user: Optional[Union[str, List[str]]] = None,
                    temperature: float = 0.7) -> Iterable[str]:
        if messages is None:
            messages = _build_messages(system, user)
        payload = {
            "model": model,
            "messages": messages,
            "temperature": temperature,
            "stream": True,
        }
        with self.sess.post(API_URL, headers=self.headers, json=payload,
                            stream=True, timeout=self.timeout) as r:
            r.encoding = "utf-8"
            r.raise_for_status()
            for raw in r.iter_lines(decode_unicode=True):
                if not raw or raw.startswith(":"):
                    continue
                data = raw[len("data: "):].strip() if raw.startswith("data: ") else raw.strip()
                if data == "[DONE]":
                    break
                try:
                    obj = json.loads(data)
                except json.JSONDecodeError:
                    continue
                chs = obj.get("choices") or []
                if not chs:
                    continue
                ch0 = chs[0]
                # 增量文本
                delta = ch0.get("delta")
                if isinstance(delta, dict) and delta.get("content"):
                    yield delta["content"]
                    continue
                # 兜底（偶发）
                msg = ch0.get("message")
                if isinstance(msg, dict) and msg.get("content"):
                    yield msg["content"]
                    continue
    
    # 在 SiliconFlowClient 类内追加该方法
    def stream_markdown(self,
                        model: str,
                        messages: Optional[List[dict]] = None,
                        system: Optional[Union[str, List[str]]] = None,
                        user: Optional[Union[str, List[str]]] = None,
                        temperature: float = 0.7,
                        update_interval: float = 0.1) -> str:
        from IPython.display import display, Markdown
        import time

        buf = []
        disp = display(Markdown(""), display_id=True)
        last = 0.0

        for chunk in self.stream_text(
            model=model,
            messages=messages,
            system=system,
            user=user,
            temperature=temperature
        ):
            buf.append(chunk)
            now = time.time()
            if now - last >= update_interval:
                disp.update(Markdown("".join(buf)))
                last = now

        final_text = "".join(buf)
        disp.update(Markdown(final_text))
        return final_text


## 初始化调用实例
## 设置使用模型、设置顶层提示词、思维链

In [2]:
client = SiliconFlowClient() 
# 或者指定路径/显式传入：
# client = SiliconFlowClient(dotenv_path="/Users/yanluohao/Learning/AI 知识分享/.env")
# client = SiliconFlowClient(api_key="你的token")

# 使用模型
usemodel="deepseek-ai/DeepSeek-V3.1"
# 系统顶级提示词
system_prompt = """“逐步解决以下问题：[问题]。”
“回答问题时，请解释每一步的推理：[问题]。”
“在提供最终答案前，请逐步思考每个步骤：[问题]。”"""

## 1.数学问题

### 同步输出结果

In [3]:
# 用户问题
user_messages = [
    "计算半径为5单位的圆的面积。请展示所有计算步骤。",
]

# 同步输出结果
text = client.complete(
    model=usemodel,
    system=system_prompt,
    user=user_messages
)
display(Markdown(text))

好的，我们一步步来计算半径为 5 单位的圆的面积。  

---

**1. 回忆圆的面积公式**  
圆的面积公式为：  
\[
A = \pi r^2
\]  
其中 \( r \) 是半径，\( \pi \) 约等于 3.14159（也可以用 \( \pi \) 的精确值保留符号）。  

---

**2. 代入半径值**  
已知 \( r = 5 \) 单位，代入公式：  
\[
A = \pi \times 5^2
\]  

---

**3. 计算半径的平方**  
\[
5^2 = 25
\]  
所以：  
\[
A = \pi \times 25
\]  
或者写成：  
\[
A = 25\pi
\]  

---

**4. 如果需要数值结果**  
取 \( \pi \approx 3.14159 \)：  
\[
A \approx 25 \times 3.14159 = 78.53975
\]  

---

**5. 最终答案**  
- 精确值：\(\boxed{25\pi}\) 平方单位  
- 近似值：\(\boxed{78.54}\) 平方单位（四舍五入到两位小数）  

---

这样，我们就完成了面积的计算。

### 流式输出结果

In [4]:
# 用户问题
user_messages = [
    "计算半径为5单位的圆的面积。请展示所有计算步骤。",
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)

好的，我们来一步一步计算半径为 5 单位的圆的面积。  

1. **圆的面积公式**  
圆的面积公式是：  
\[
A = \pi r^2
\]
其中 \( r \) 是圆的半径，\( \pi \) 通常取值约 3.14159。

2. **代入半径值**  
已知半径 \( r = 5 \) 单位，代入公式：  
\[
A = \pi \times (5)^2
\]

3. **计算半径的平方**  
\[
5^2 = 25
\]
所以：
\[
A = \pi \times 25
\]

4. **最终结果**  
\[
A = 25\pi
\]
如果用数值近似（取 \(\pi \approx 3.1416\)）：
\[
A \approx 25 \times 3.1416 = 78.54
\]

**答案：**  
\[
\boxed{25\pi}
\]
近似值为 \( 78.54 \) 平方单位。

## 2.逻辑推理

In [5]:
# 用户问题
user_messages = [
    "以下三段论是否有效？所有哺乳动物是温血动物。所有鲸鱼是哺乳动物。因此，所有鲸鱼是温血动物。请解释你的推理过程。"
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)

好的，我们一步步推理。  

**1. 识别三段论的结构**  
- 大前提：所有哺乳动物是温血动物。（M 是 P）  
- 小前提：所有鲸鱼是哺乳动物。（S 是 M）  
- 结论：所有鲸鱼是温血动物。（S 是 P）  

其中：  
- S = 鲸鱼  
- M = 哺乳动物  
- P = 温血动物  

**2. 检查推理形式**  
这是典型的**三段论第一格 AAA 式（Barbara 式）**：  
- 所有 M 是 P  
- 所有 S 是 M  
- 所以，所有 S 是 P  

这个形式在逻辑上是**有效**的。  
因为如果 M 全部包含在 P 中，而 S 全部包含在 M 中，那么 S 必然全部包含在 P 中。  

**3. 验证内容是否符合形式**  
现实世界中，前提为真（事实上所有哺乳动物是温血动物，所有鲸鱼是哺乳动物），但逻辑有效性只依赖于形式，不依赖于内容真假。  

**4. 结论**  
该三段论**有效**。

# 二、语言桥接技术 ( Language Bridging )

In [50]:
# 更换语言桥接技术提示词
system_prompt = """
你是一个语言桥接专家，请将以下问题翻译成中文，并给出详细的推理过程，最终给出翻译结果。
"""

## 示例一：查询日本传统文化

In [51]:
# 用户问题
user_messages = [
    "歌舞伎の歴史とその文化的な重要性について詳しく教えてください。"
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)

请详细告诉我歌舞伎的历史及其文化重要性。  

推理过程：  
1. 识别原文语言：原文为日语，包含关键词“歌舞伎”（Kabuki）、“歴史”（历史）和“文化的な重要性”（文化重要性）。  
2. 理解句子结构：句子是请求详细解释歌舞伎历史和文化重要性的疑问句，语气礼貌（使用“ください”）。  
3. 转换到中文：  
   - 将“歌舞伎”直接译为中文专用名词“歌舞伎”。  
   - “歴史”译为“历史”，“文化的な重要性”译为“文化重要性”。  
   - 调整语序和礼貌表达：日语疑问句常以“ください”结尾，中文用“请...告诉我”保持礼貌，并以句号结束。  
4. 确保流畅性：中文表达需自然，避免直译生硬，因此使用“详细告诉我”来对应“詳しく教えてください”。  

翻译结果：  
请详细告诉我歌舞伎的历史及其文化重要性。

## 示例二：医学领域的专业查询

In [52]:
# 用户问题
user_messages = [
    "Erklären Sie die Pathophysiologie der chronischen myeloischen Leukämie."
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)

请解释慢性粒细胞白血病的病理生理学。

推理过程：
1. 识别源语言：该问题为德语，需要翻译成中文。
2. 分析句子结构：原句是一个完整的陈述句，要求解释慢性粒细胞白血病的病理生理学。
3. 关键术语翻译：
   - "Erklären Sie" 对应中文的"请解释"（礼貌性指令）
   - "die Pathophysiologie" 译为"病理生理学"（医学术语）
   - "der chronischen myeloischen Leukämie" 译为"慢性粒细胞白血病"（标准医学译名）
4. 文化适应性调整：中文医学语境中通常使用完整术语"慢性粒细胞白血病"，而非缩写形式。
5. 句式重组：德语动词"Erklären"位于句首的祈使句式，转换为中文更自然的"请解释..."礼貌句式。

翻译结果：请解释慢性粒细胞白血病的病理生理学。

# 元提示（ Meta Prompt ）
元提示提升 GPT 互动效果

In [54]:
# 普通提示示例
# 更换语言桥接技术提示词
system_prompt = """
"""
# 用户问题
user_messages = [
    "解释光合作用的过程。"
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)

好的，我们来详细解释一下光合作用的过程。

光合作用是植物、藻类和某些细菌利用光能，将二氧化碳和水转化为有机物（如葡萄糖）并释放氧气的过程。它是地球上几乎所有生命赖以生存的基础。

其总反应式可以概括为：
**6CO₂ + 6H₂O + 光能 → C₆H₁₂O₆ (葡萄糖) + 6O₂**

这个过程主要发生在植物的**叶绿体**中，更具体地说，是在叶绿体内的**类囊体膜**和**基质**中。光合作用可以分为两个主要阶段：**光反应**和**暗反应**（又称卡尔文循环）。

---

### 第一阶段：光反应

光反应必须在有光的条件下进行，发生在**类囊体膜**上。

1.  **光能的吸收**：叶绿体中的色素（主要是叶绿素a和b）捕捉太阳光的光子（能量包）。
2.  **水的光解**：捕获的光能用于分解水分子。这个过程被称为“光解”。
    *   **方程式**： 2H₂O → 4H⁺ + 4e⁻ + O₂
    *   **产物**：
        *   **氧气 (O₂)**：作为副产品被释放到大气中，这是我们呼吸的氧气来源。
        *   **高能电子 (e⁻)**：被激发到高能状态，进入电子传递链。
        *   **质子 (H⁺)**：被泵入类囊体腔，形成浓度梯度。
3.  **能量载体的合成**：高能电子在类囊体膜上的电子传递链中传递，这个过程就像下楼梯一样，逐步释放能量。释放的能量有两个重要作用：
    *   **合成 ATP**：利用质子梯度驱动ATP合酶工作，将ADP和磷酸合成**ATP**（腺苷三磷酸）。这个过程称为**光合磷酸化**。ATP是细胞的“能量货币”。
    *   **合成 NADPH**：最终，电子和质子（H⁺）被传递给一种叫做NADP⁺的辅酶，形成**NADPH**。NADPH是一种强还原剂，携带者“还原力”。

**光反应小结**：
*   **地点**：类囊体膜
*   **输入**：光能、水（H₂O）
*   **输出**：**ATP**（能量）、**NADPH**（还原力）、**氧气 (O₂)**（废物）

---

### 第二阶段：暗反应（卡尔文循环）

暗反应可以在有光或无光的条件下进行，但它依赖于光反应产生的ATP和NADPH，因此通常发生在白天。它发生在叶绿体的**基质**中。

这个阶段不再直接需要光，而是利用光反应中产生的能量（ATP和NADPH）来固定二氧化碳，合成有机物。

1.  **碳固定**：一个叫做**RuBisCO**的关键酶，捕获空气中的二氧化碳（CO₂），并将其固定在一个五碳化合物（RuBP，1,5-二磷酸核酮糖）上。形成一个不稳定的六碳中间体，然后迅速分解成两个三碳化合物（3-磷酸甘油酸，PGA）。
2.  **还原阶段**：利用光反应产生的**ATP**和**NADPH**的能量和还原力，将PGA还原成另一种三碳糖——甘油醛-3-磷酸（G3P）。NADPH提供氢原子，ATP提供能量。
3.  **再生**：大部分G3P被用来再生最初的二氧化碳受体RuBP，以便循环可以继续。这个过程也需要消耗ATP。
4.  **糖的合成**：每进行6次卡尔文循环，可以净产出1个G3P分子。这些G3P分子是碳骨架，可以被运出叶绿体，用于合成**葡萄糖**、**蔗糖**、**淀粉**、**纤维素**等更复杂的有机物，为植物提供能量和结构物质。

**暗反应小结**：
*   **地点**：叶绿体基质
*   **输入**：二氧化碳（CO₂）、ATP、NADPH
*   **输出**：有机物（如葡萄糖）、再生后的RuBP
*   **关键酶**：RuBisCO（地球上含量最丰富的酶）

---

### 总结与比喻

您可以把光合作用想象成一个**绿色工厂**：

*   **工厂地址**：叶绿体
*   **动力车间（光反应）**：太阳能电池板（色素）吸收阳光，驱动机器分解水，生产出电能（ATP）和电池（NADPH），并排出氧气废气。
*   **组装车间（暗反应）**：利用动力车间送来的电能和电池，将原材料二氧化碳组装成糖类产品。

这个过程不仅是植物自身生长的基础，也构成了食物链的起点，并维持了大气中氧气和二氧化碳的平衡，对地球生态系统至关重要。

In [55]:
# 元提示示例
# 更换语言桥接技术提示词
system_prompt = """
"""
# 用户问题
user_messages = [
    "你是一位生物学老师，请用适合12岁学生理解的方式，逐步解释光合作用。"
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)

当然可以！我会像在课堂上一样，用简单有趣的方式来解释光合作用。让我们一步步来。

---

### 🌱 第一步：植物也需要“吃饭”吗？

想象一下，你中午肚子饿了，会吃饭来获取能量。植物也需要能量来生长，但它们不能像我们一样去吃东西。那它们怎么办？  
**答案：它们自己制作“食物”！** 这个过程就叫做**光合作用**。

---

### ☀️ 第二步：神奇的“厨房”——叶子

植物的叶子就像一个小小的“厨房”，里面有无数个叫**叶绿体**的“迷你厨师”。  
*   **叶绿体**是绿色的（所以叶子是绿色的），它里面有一种叫**叶绿素**的物质，可以吸收太阳光，就像太阳能板一样。

---

### 🧪 第三步：制作食物的“配方”

要制作食物，我们的“迷你厨师”需要三种原料：

1.  **💧 水**：植物用根从土壤里吸收水分。
2.  **🌬️ 二氧化碳**：植物通过叶子背面微小的气孔，从空气中吸收二氧化碳（就是我们呼出来的气体）。
3.  **☀️ 阳光**：太阳提供能量。

---

### 🍳 第四步：厨房开工啦！

有了原料和能源，厨房就开始工作了！  
在叶绿体里，阳光的能量被用来把**水**和**二氧化碳**混合在一起，发生神奇的化学反应。

**这个反应会制造出两种东西：**

1.  **🍯 葡萄糖**：这是一种糖，就是植物的“食物”。植物用它来生长、开花、结果。
2.  **💨 氧气**：这个过程还会产生氧气，并从叶子的气孔释放到空气中。

**所以，光合作用就像一个神奇的食谱：**

**水 + 二氧化碳 + 阳光能量 → 葡萄糖（食物） + 氧气**

---

### 🌍 第五步：为什么光合作用超级重要？

光合作用不仅仅对植物重要，对我们和整个地球都至关重要！

1.  **制造氧气**：它为我们提供了呼吸所需要的氧气。没有植物，我们就没法呼吸！
2.  **提供食物**：它制造的食物是所有动物（包括我们人类）直接或间接的能量来源。我们吃的蔬菜、水果、粮食，甚至吃草的牛，都来自光合作用。
3.  **清洁空气**：它能吸收我们呼出的二氧化碳，帮助净化空气。

---

### ✨ 总结一下：

你可以把光合作用想象成植物在用阳光做饭！

*   **地点**：叶子的叶绿体（厨房）
*   **厨师**：叶绿素
*   **原料**：水、二氧化碳
*   **能源**： sunlight（阳光）
*   **做出的产品**：葡萄糖（食物）和氧气（废气，但对我们是宝贝！）

下次你看到一棵树或一株小草，记得它们正在默默地为我们制造氧气和食物哦！是不是很厉害？

# 反向交互 （ Flipped Interaction ）

### 示例1：网络故障诊断

In [ ]:
# 更换提示词
system_prompt = """

"""
# 用户问题
user_messages = [
    "请通过向我提问帮助我诊断我的网络问题。提问直到你收集到足够的信息，找出最可能的两个原因。一次问一个问题。现在请问第一个问题。"
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)

# 用户回答：  
# 所有设备都无法连接。


好的，我们一步步来排查问题。

首先，请告诉我：**你的设备是完全无法连接到网络，还是可以连接但网速很慢/不稳定？**

### 示例2：教育辅导

In [57]:
# 更换提示词
system_prompt = """

"""
# 用户问题
user_messages = [
    "请通过向我提问帮助我学习微积分的基本概念。提问直到你了解我对微积分的掌握程度。一次问一个问题。现在请问第一个问题。"
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)

# 用户回答：  
# 所有设备都无法连接。


好的，我们开始吧。第一个问题：

**微积分主要分为哪两个分支？它们分别研究什么？**

# 少样本学习 （ Few-shot Learning ）

In [ ]:
# 更换提示词
system_prompt = """
将英语句子翻译成法语。
输出示例：
*“Hello, how are you?” → “Bonjour, comment ça va ?”*
*“I would like a cup of coffee.” → “Je voudrais une tasse de café.”*
"""
# 用户问题
user_messages = [
    "Hello, how are you? → Bonjour, comment ça va ? I would like a cup of coffee. → Je voudrais une tasse de café. I am going to the market. →"
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



Je vais au marché.

In [59]:
# 更换提示词
system_prompt = """
将段落总结为一句话。

"""
# 用户问题
user_messages = [
    """1. 段落：'人工智能正在通过使机器能够执行通常需要人类智能的任务而改变行业，包括语音识别、决策和视觉感知。'  
总结：'人工智能使机器能够执行人类任务，从而改变行业。'

2. 段落：'气候变化导致更频繁的极端天气事件。气温上升引发了严重的风暴、干旱和热浪，这些都影响了生态系统和人类社会。'  
总结：'气候变化引发极端天气事件，影响生态系统和社会。'

3. 段落：'电动车因技术进步和环境问题的关注而越来越受欢迎。它们为传统燃油汽车提供了可持续的替代方案。'  
总结："""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



电动车因技术进步和环保需求而日益普及，成为传统燃油车的可持续替代品。

In [60]:
# 更换提示词
system_prompt = """
分析顾客评论的情感倾向。
"""
# 用户问题
user_messages = [
    """
    分析以下评论的情感倾向：
1. '产品质量非常好，超出了我的预期。' → 积极
2. '我对服务感到失望，速度非常慢。' → 消极
3. '体验一般，没有特别之处。' → 中立
4. '我非常喜欢这个！还会再次购买。' →
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



积极

In [61]:
# 更换提示词
system_prompt = """
将下列的时间数据改成“YYYY-MM-DD”格式:
"""
# 用户问题
user_messages = [
    """
   输入:  
03/15/2023  
12/01/2022  
输出:  
2023-03-15  
2022-12-01  

输入:  
07/04/2021  
11/11/2020  

输出: 
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



2021-07-04  
2020-11-11

# 游戏模式 （ Game Model ）

In [62]:
# 示例1：洞穴探险游戏寻找失落的语言
# 更换提示词
system_prompt = """
为我设计一个洞穴探险游戏，以发现失落的语言。描述我在洞穴中的位置和可以采取的行动。在每个区域，我都应该发现新的单词和符号，每个区域还包含用该语言讲述的一部分故事。最终，我需要收集所有单词和符号，才能理解完整的故事。描述第一个区域并询问我想采取什么行动。
"""
# 用户问题
user_messages = [
    """
   开始游戏
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



你站在一个巨大洞穴的入口处，湿冷的空气带着泥土和岩石的气息扑面而来。前方，一条狭窄的通道蜿蜒深入黑暗，洞壁上刻着奇怪的符号，似乎是一种未知的语言。微弱的光线从洞顶的裂缝中透下，照亮了入口附近的区域。

在你面前的石壁上，你注意到一组符号，旁边似乎有简单的图画：一个弯曲的线条可能代表“洞穴”，一个尖锐的符号可能代表“入口”。这是你遇到的第一个单词片段。

**当前位置：洞穴入口**  
**已收集：无**  

你可以：  
- 检查符号（尝试解读）  
- 向前进入通道  
- 搜索周围区域  

你想采取什么行动？

In [63]:
# 示例1：洞穴探险游戏寻找失落的语言
# 更换提示词
system_prompt = """
我们来做中文诗歌游戏。 你选一首诗歌， 你出上句， 我回答下一句。 你告诉我的答案对错。 如果我答错了， 你给我一次重试的机会。 如果我再次错了， 你就告诉我正确答案。 然后继续游戏。
"""
# 用户问题
user_messages = [
    """
   开始游戏
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



好的，让我们开始吧！第一句是：  
**床前明月光，**  
请接下句。

# 提示词完善 （ Prompt Refinement ）

In [65]:
# 更换提示词
system_prompt = """
为用户重新创建更详细的提示词。并使用新的提示词重新回答问题。
在回答前给用户展示你重新创建的提示词内容。
"""
# 用户问题
user_messages = [
    """
   告诉我工业革命的原因和影响。
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



### 重新创建的提示词
为了提供更全面的回答，以下是我基于你的原始查询重新创建的详细提示词：
- **主题范围**：工业革命（18世纪至19世纪，主要聚焦英国，兼顾全球背景）。
- **具体要求**：
  1. 解释工业革命的主要原因，包括经济、技术、社会和政治因素。
  2. 分析工业革命对经济、社会、环境和文化的影响，包括正面和负面方面。
  3. 提供具体例子（如蒸汽机、纺织业变革、城市化）。
  4. 结构清晰，分点列出原因和影响。
  5. 语言简洁易懂，适合普通读者。

---

### 回答
工业革命（约1760-1840年）是近代历史上从农业社会向工业社会转型的关键时期，起源于英国，后扩散至全球。以下是其**原因**和**影响**的详细分析：

#### 原因
工业革命的发生是多种因素交织的结果：
1. **经济因素**：
   - **资本积累**：英国通过殖民扩张和贸易（如三角贸易）积累了大量资本，可用于投资新技术。
   - **市场需求增长**：人口增加和海外殖民扩张导致对商品（如纺织品）的需求上升，传统手工业无法满足。
2. **技术因素**：
   - **关键发明**：蒸汽机（詹姆斯·瓦特改进）、纺织机械（如纺纱机和动力织机）提高了生产效率。
   - **能源革命**：煤炭取代木材成为主要能源，支持了工厂的大规模运营。
3. **社会与政治因素**：
   - **农业革命**：圈地运动提高了农业效率，释放了大量农村劳动力，为工厂提供工人。
   - **政治稳定**：英国相对稳定的政府和法律体系（如专利法）鼓励了创新和投资。
4. **资源与地理**：英国拥有丰富的煤炭和铁矿石资源，以及便利的运河和港口网络，便于运输原材料和成品。

#### 影响
工业革命带来了深远的变化，既有积极也有消极方面：
- **经济影响**：
  - **正面**：生产效率大幅提升，经济增长加速，资本主义经济体系确立。例如，英国成为“世界工厂”，GDP显著增长。
  - **负面**：早期工人贫困化，经济不平等加剧，周期性的经济危机出现（如1837年恐慌）。
- **社会影响**：
  - **正面**：城市化进程加速（如曼彻斯特和伦敦扩张），中产阶级兴起，生活水平总体长期提高。
  - **负面**：工人阶级生活条件恶劣（长时间工作、低工资、童工问题），贫富差距扩大，社会冲突频发（如卢德运动）。
- **环境影响**：
  - **负面**：污染加剧（空气和水污染），自然资源过度开采， deforestation（森林砍伐）问题显现，为现代环境问题埋下伏笔。
- **文化与全球影响**：
  - **技术进步**：推动了科学和教育发展，如铁路和电报的发明改变了通信和交通。
  - **殖民扩张**：工业国家（如英国、法国）利用工业优势进行全球殖民，导致全球化加速，但也引发了不平等的国际关系。

总结：工业革命是人类历史上的转折点，它奠定了现代工业社会的基础，但同时也带来了社会和环境挑战。其遗产至今仍在影响全球经济发展和可持续性讨论。如果需要更深入的内容，我可以进一步扩展！

# 提示词深化 （ Prompt Verifier ）

In [66]:
# 更换提示词
system_prompt = """
每当你被问到一个问题时，请遵循以下的规则：

1. 请生成一些额外的问题，这些问题将帮助更准确地回答这个问题。

2. 将对这些单独问题的回答结合起来，以产生对整个问题的最终答案。
-----
输出你生成的额外子问题，以及对这些额外子问题的回答。
"""
# 用户问题
user_messages = [
    """
现在全世界面临的气候问题是什么？
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



### 额外问题：
1. 全球气候变化的主要驱动因素是什么？
2. 气候变化对环境和人类社会的主要影响有哪些？
3. 当前国际社会在应对气候变化方面的主要努力和挑战是什么？

### 对额外问题的回答：
1. **全球气候变化的主要驱动因素**：主要包括人为活动，如燃烧化石燃料（煤、石油、天然气）导致的温室气体排放（尤其是二氧化碳和甲烷），以及森林砍伐和土地利用变化。这些因素增强了温室效应，导致全球变暖。自然因素（如火山活动和太阳辐射变化）也有贡献，但相比人为因素较小。

2. **气候变化对环境和人类社会的主要影响**：环境方面包括极端天气事件（如热浪、洪水、干旱）频率和强度增加、海平面上升、冰川融化、生物多样性丧失和生态系统破坏。人类社会方面包括粮食安全威胁（农业减产）、水资源短缺、健康风险（如热相关疾病和传染病传播）、经济成本增加（如基础设施损坏）以及人口迁移和冲突风险上升。

3. **当前国际社会在应对气候变化方面的主要努力和挑战**：主要努力包括《巴黎协定》（目标是将全球变暖控制在2°C以内，并努力限制在1.5°C）、国家减排承诺（如NDCs）、可再生能源推广（太阳能、风能）、碳定价机制和国际合作（如COP会议）。挑战包括各国承诺不足、资金和技术转移问题（尤其对发展中国家）、政治意愿缺乏、以及适应措施的滞后。

### 最终答案：
全世界面临的气候问题主要是由人为温室气体排放驱动的全球气候变化，表现为全球变暖、极端天气事件频发、海平面上升和生态系统退化等。这些问题对环境和人类社会造成广泛负面影响，包括粮食安全、健康和经济风险。国际社会通过《巴黎协定》等努力应对，但面临承诺不足、资金短缺和政治挑战等障碍，亟需加强全球合作和行动以减缓并适应气候变化。

# 近期偏差（Recency Bias）

In [ ]:
# 不太有效的提示词
# 更换提示词
system_prompt = """

"""
# 用户问题
user_messages = [
    """
请写一封专业的电子邮件。开头问候，正文介绍会议信息，结尾礼貌结束。
P.S. 语气要轻松随和。
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



**主题：邀请参加本周团队会议**  

嗨，大家好！  

希望这封邮件能让你拥有一个愉快的一天！  

本周五（[日期]）上午10点，我们将召开一次团队会议，地点在[会议室名称/线上会议链接]。这次会议的主要目的是讨论近期的项目进展，并规划下一阶段的工作安排。如果你有任何想要分享的议题或需要添加到议程中的内容，请随时提前告诉我。  

期待与大家见面，一起交流想法！如果时间安排上有任何问题，也请尽快告知我。  

祝好，  
[你的名字]  
[你的职位]  
[联系方式]  

---  
**P.S.** 会议结束后我们会准备一些茶点，欢迎大家留下来轻松交流哦 😊

In [69]:
# 有效的提示词
# 更换提示词
system_prompt = """

"""
# 用户问题
user_messages = [
    """
请写一封专业且正式的电子邮件。P.S. 请确保语气专业且正式。
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



**主题：请求协助更新项目时间表**

尊敬的[收件人姓名]：  

您好！  

希望此邮件送达时您一切顺利。  

我写信是希望请求您协助更新[项目名称]的时间表。为确保项目按计划推进，我们需要对当前的时间节点进行复核，并根据最新进展进行相应调整。  

如果您能提供最新的项目进展信息，并在[具体日期]前反馈给我，我将不胜感激。如有任何疑问或需要进一步讨论，请随时与我联系。  

感谢您的支持与配合！  

此致  
敬礼  

[您的姓名]  
[您的职位]  
[您的联系方式]  
[日期]  

---

**P.S.** 如果您需要进一步的信息或协助，请随时告知。

# 配方模式 （ Recipe Model ）

In [71]:
# 有效的提示词
# 更换提示词
system_prompt = """
你是一个商业顾问，帮助用户制定商业计划和策略。
"""
# 用户问题
user_messages = [
    """
我公司要开发一个民用的车用清洁剂。 我的产品原型已经完成，市场上有类似的竞争产品。
请告诉我如何获得商业成功.
请为我提供完整的步骤序列，填补任何缺失的步骤。
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



以下是帮助您的民用清洁剂产品在竞争激烈的市场中取得商业成功的完整步骤序列。该计划分为七个阶段，覆盖从市场调研到持续优化的全过程：

### 第一阶段：市场调研与竞争分析（1-2个月）
1. **目标市场细分**：确定您的目标客户（例如私家车主、汽车美容店、环保意识强的消费者）。
2. **竞争对手分析**：识别主要竞争产品（如3M、龟牌等），分析其定价、成分、销售渠道、营销策略和用户评价。
3. **差异化定位**：基于您的产品原型，找出独特卖点（如环保配方、更强去污力、低成本、使用便捷性）。
4. **用户需求验证**：通过问卷调查、焦点小组或试用活动，收集潜在用户的反馈，确认产品是否满足市场需求。

### 第二阶段：产品优化与合规准备（1-2个月）
1. **配方与包装改进**：根据市场反馈优化产品性能（如清洁效果、安全性），设计吸引人的包装（环保材料、易用性）。
2. **法规合规**：确保产品符合国家化学品安全标准（如MSDS认证）、环保标志（如可生物降解认证）以及标签法规。
3. **小规模生产测试**：生产一批试用品，进行实地测试（如与汽车美容店合作），收集性能数据。

### 第三阶段：商业模式与定价策略（1个月）
1. **成本分析**：计算生产成本、包装、物流和营销费用，确定盈亏平衡点。
2. **定价策略**：基于竞争产品定价（中端或高端定位），考虑渗透定价（低价进入市场）或价值定价（强调差异化）。
3. **销售渠道规划**：选择线上线下渠道（如电商平台（淘宝、京东）、汽车用品店、4S店合作、社交媒体直销）。

### 第四阶段：品牌建设与营销策略（2-3个月）
1. **品牌 identity**：开发品牌名称、logo和口号，强调差异化（如“高效环保清洁专家”）。
2. **内容营销**：创建使用教程、对比测评视频（展示与竞品的优势），通过抖音、小红书、微信公众号传播。
3. **合作伙伴关系**：与汽车博主、美容店、4S店合作进行推广或分销，提供样品换评测或分成协议。
4. **促销活动**：推出上市优惠（如买一送一）、限时折扣，或捆绑销售（搭配其他汽车护理产品）。

### 第五阶段：销售与分销启动（持续进行）
1. **电商平台上架**：在主流平台开设旗舰店，优化产品列表（关键词、高质量图片、用户评价管理）。
2. **线下渠道拓展**：与零售商或汽车服务商签订分销协议，提供铺货支持。
3. **客户服务系统**：设置客服团队处理咨询和售后，确保 positive 用户体验。

### 第六阶段：性能监控与反馈循环（持续进行）
1. **销售数据跟踪**：监控销售渠道表现，分析哪些渠道或促销活动最有效。
2. **用户反馈收集**：通过评论、调查和社交媒体监听，持续改进产品。
3. **迭代优化**：根据反馈调整配方、包装或营销策略，保持竞争力。

### 第七阶段：规模化与扩张（6-12个月后）
1. **扩大生产**：根据需求增加产能，考虑OEM或自有工厂。
2. **产品线扩展**：开发相关产品（如内饰清洁剂、玻璃水），提升客户生命周期价值。
3. **市场扩张**：进入新区域或国际市场，复制成功模式。

### 关键成功因素：
- **差异化**：强调产品独特之处（如环保、性价比），避免同质化竞争。
- **客户参与**：通过试用和互动建立忠诚度。
- **敏捷响应**：根据市场变化快速调整策略。

这个序列覆盖了从准备到增长的全过程。如果您需要更详细的某一步骤（如营销策略或合规细节），我可以提供进一步指导。祝您的产品成功！

# 自然语言编程 - 缩写 （Abbreviation）

In [72]:
# 有效的提示词
# 更换提示词
system_prompt = """
当我说 主题（“分钟”， <前置主题>， [后置主题]），就意味着：
- 这个主题需要占用的时间为“分钟”。
- 这个主题需要在 <前置主题> 完成后发生。
- 这个主题需要在 [后置主题] 发生前完成。
"""
# 用户问题
user_messages = [
    """
请为我安排会议日程，其中需要包括的主题有：
- 签到（“30”）
- 开幕式（“15”， <签到>）
- 市领导讲话（“30”， <开幕式>）
- 主管局领导讲话（“30”， <市领导讲话>）
- 协会领导讲话（“30”， <主管局领导讲话>）
- 专家部分（“0”， <协会领导讲话>）
- 国际专家讲话（“40”， <专家部分>， [项目部分]）
- 国内专家1讲话（“30”， <专家部分>， [项目部分]）
- 国内专家2讲话（“45”， <专家部分>， [项目部分]）
- 项目部分（“0”， [项目进度汇报]）
- 项目进度汇报（“60”， <项目部分>， [颁奖仪式]）
- 大会总结（“30”， [会议结束]）
- 颁奖仪式（“30”， <项目进度汇报>， [大会总结]）
- 会议结束（“5”， <大会总结>)
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



根据您提供的主题及其时间、前置和后置关系，我将为您安排一个详细的会议日程。以下是按时间顺序排列的会议议程，包括每个主题的开始和结束时间，以及总会议时长。

### 会议日程安排
**总时长：约 5 小时 30 分钟（从开始到结束）**

假设会议从 **09:00** 开始，以下是具体时间表：

1. **签到**  
   - 时间：09:00 - 09:30（30分钟）  
   - 前置：无  
   - 后置：开幕式  

2. **开幕式**  
   - 时间：09:30 - 09:45（15分钟）  
   - 前置：签到  
   - 后置：市领导讲话  

3. **市领导讲话**  
   - 时间：09:45 - 10:15（30分钟）  
   - 前置：开幕式  
   - 后置：主管局领导讲话  

4. **主管局领导讲话**  
   - 时间：10:15 - 10:45（30分钟）  
   - 前置：市领导讲话  
   - 后置：协会领导讲话  

5. **协会领导讲话**  
   - 时间：10:45 - 11:15（30分钟）  
   - 前置：主管局领导讲话  
   - 后置：专家部分（过渡环节）  

6. **专家部分（过渡环节）**  
   - 时间：11:15 - 11:15（0分钟，仅作为逻辑节点）  
   - 前置：协会领导讲话  
   - 后置：国际专家讲话、国内专家1讲话、国内专家2讲话（这些讲话可以并行或顺序进行，但需在项目部分之前完成）  
   - 注意：由于专家部分本身不占用时间，它只是一个分组标记。实际安排中，专家讲话可以按顺序进行，总时长取决于具体讲话。

   根据您提供的约束，所有专家讲话必须在项目部分开始前完成。因此，我将专家讲话顺序安排如下（总时长115分钟）：
   - **国际专家讲话**：11:15 - 11:55（40分钟）
   - **国内专家1讲话**：11:55 - 12:25（30分钟）
   - **国内专家2讲话**：12:25 - 13:10（45分钟）

   但请注意，这会导致会议在中午连续进行，可能需安排午餐休息。如果您希望避免中断，可以考虑调整顺序或加入休息时间。这里我假设会议连续进行。

7. **项目部分（过渡环节）**  
   - 时间：13:10 - 13:10（0分钟，仅作为逻辑节点）  
   - 前置：所有专家讲话完成  
   - 后置：项目进度汇报  

8. **项目进度汇报**  
   - 时间：13:10 - 14:10（60分钟）  
   - 前置：项目部分  
   - 后置：颁奖仪式  

9. **颁奖仪式**  
   - 时间：14:10 - 14:40（30分钟）  
   - 前置：项目进度汇报  
   - 后置：大会总结  

10. **大会总结**  
    - 时间：14:40 - 15:10（30分钟）  
    - 前置：颁奖仪式  
    - 后置：会议结束  

11. **会议结束**  
    - 时间：15:10 - 15:15（5分钟）  
    - 前置：大会总结  
    - 后置：无  

### 总时间线摘要
- 09:00 - 09:30: 签到
- 09:30 - 09:45: 开幕式
- 09:45 - 10:15: 市领导讲话
- 10:15 - 10:45: 主管局领导讲话
- 10:45 - 11:15: 协会领导讲话
- 11:15 - 11:55: 国际专家讲话
- 11:55 - 12:25: 国内专家1讲话
- 12:25 - 13:10: 国内专家2讲话
- 13:10 - 14:10: 项目进度汇报
- 14:10 - 14:40: 颁奖仪式
- 14:40 - 15:10: 大会总结
- 15:10 - 15:15: 会议结束

### 注意事项
- **午餐安排**：专家讲话部分从11:15持续到13:10，跨越午餐时间。建议在日程中加入午餐休息（例如，在专家讲话后安排30-60分钟休息），否则参会者可能疲劳。如果需要，我可以调整日程。
- **灵活性**：实际执行中，讲话可能略有超时或提前结束，请预留缓冲时间。
- **并行环节**：专家讲话被设计为顺序进行，但如果您希望并行（例如分会场），需调整后置关系。

如果您需要加入休息或调整时间，请提供更多细节！

# 自然语言编程 - 函数 （ Function ）

In [73]:
# 示例 1：翻译
# 更换提示词
system_prompt = """
- 函数名称：[翻译]
- 输入：[需要翻译的文本]
- 规则：[将输入文本从英语翻译成法语，保持含义和语气。]
- 输出：[翻译后的法语文本]

"""
# 用户问题
user_messages = [
    """
翻译("Hello, how are you?")
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



("Bonjour, comment allez-vous ?")

In [74]:
# 示例 2：文本摘要
# 更换提示词
system_prompt = """
- 函数名称：[摘要]
- 输入：[需要总结的文本]
- 规则：[将输入文本总结为一句简洁的句子。]
- 输出：[一句话的摘要]


"""
# 用户问题
user_messages = [
    """
摘要("人工智能使机器能够模仿人类的能力，例如学习和推理。")
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



人工智能使机器能够模仿人类的学习与推理能力。

In [ ]:
# 示例 3：数据提取
# 更换提示词
system_prompt = """
- 函数名称：[提取信息]
- 输入：[包含详细信息的结构化文本]
- 规则：[从输入文本中提取订单号和金额。]
- 输出：[提取的内容，格式为“订单号：[编号]，金额：[数值]”]
"""
# 用户问题
user_messages = [
    """
提取信息("订单号：12345，日期：2024-12-06，金额：250美元")
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



订单号：12345，金额：250美元

# 自然语言编程 - 控制 （ Control ）


## If/Then/Else 条件控制

In [ ]:
# 示例 1：简单的数值判断
# 更换提示词
system_prompt = """
你得到一个数字：[用户输入]。
如果数字大于 0，写“正数”。
否则，写“非正数”。
"""
# 用户问题
user_messages = [
    """
10
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)



正数

In [ ]:
# 示例 2：基于文本内容的判断
# 更换提示词
system_prompt = """
你将收到一条用户消息。如果消息中包含“帮助”一词，回复“我能帮您什么？”否则，回复“好的，已记录。
"""
# 用户问题
user_messages = [
    """
请帮我解决登录问题。
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)

# 用户消息：“今天的任务已经完成。”

我能帮您什么？

## For 循环控制

In [78]:
# 示例 1：固定次数的重复
# 更换提示词
system_prompt = """
将短语“你好，世界”重复 3 次，每次占一行。
"""
# 用户问题
user_messages = [
    """
请帮我解决登录问题。
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)


我目前无法处理登录相关的技术问题。但你可以尝试以下步骤：

1️⃣ 检查网络连接是否正常
2️⃣ 确认用户名和密码是否正确
3️⃣ 尝试重置密码或找回账号
4️⃣ 清除浏览器缓存或更换浏览器尝试
5️⃣ 联系相关平台的技术支持

如果还有其他我能帮助的问题，请随时告诉我！

In [79]:
# 示例 2：遍历列表中的元素并转换为拼音
# 更换提示词
system_prompt = """
从用户给定列表 []，对列表中的每个单词，将其转为拼音并换行显示。
"""
# 用户问题
user_messages = [
    """
[“苹果”, “香蕉”, “樱桃”]
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)


苹果：píng guǒ  
香蕉：xiāng jiāo  
樱桃：yīng táo

In [80]:
# 示例 3：为中文单词添加标注
# 更换提示词
system_prompt = """
给定列表 []，对列表中的每个单词，在后面添加“是一种水果”的标注，并换行显示。
"""
# 用户问题
user_messages = [
    """
[“苹果”, “香蕉”, “樱桃”]
"""
]

# 流式渲染（逐步更新 Markdown）
final_text = client.stream_markdown(
    model=usemodel,
    system=system_prompt,
    user=user_messages,
    update_interval=0.1
)


苹果是一种水果  
香蕉是一种水果  
樱桃是一种水果